In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
import datetime
from vars import path

In [ ]:
dataset_path = path

log_dir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    labels="inferred",
    label_mode="categorical",
    seed=123,
    image_size=(200, 200),
    batch_size=64)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    labels="inferred",
    label_mode="categorical",
    seed=123,
    image_size=(200, 200),
    batch_size=64)

class_names = train_dataset.class_names

train_dataset = train_dataset.map(lambda x, y: (x/255, y))
validation_dataset = validation_dataset.map(lambda x, y: (x/255, y))

AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, GaussianNoise
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom, RandomTranslation


base_model = MobileNetV2(include_top=False,
                            weights='imagenet')

base_model.trainable = False

model = Sequential([
    RandomFlip("horizontal", input_shape=(150, 150, 3)),
    RandomRotation(0.1),
    GaussianNoise(0.1),
    BatchNormalization(renorm=True),
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.2),
    Dense(100, activation='softmax')
])

model.summary()

In [4]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_dataset,
    epochs=12,
    validation_data=validation_dataset)

In [6]:
model.save('MobileNetV2Gaussian.keras')

In [ ]:
predictions = model.predict(validation_dataset)

In [ ]:
test = model.predict(validation_dataset)

In [ ]:
test

In [10]:
top3label = tf.math.top_k(test, k=3).indices.numpy()


print(len(top3label))

1975


In [11]:
t3labels = [[class_names[index] for index in prediction] for prediction in top3label]

print(t3labels)

[['Cattle_Egret', 'Black_legged_Kittiwake', 'Caspian_Tern'], ['Bushtit', 'Carolina_Chickadee', 'Black_capped_Chickadee'], ['Blackpoll_Warbler', 'Black_throated_Green_Warbler', 'Blue_gray_Gnatcatcher'], ['American_Redstart', 'Black_backed_Woodpecker', 'Cassins_Sparrow'], ['Brown_Creeper', 'Cattle_Egret', 'Carolina_Chickadee'], ['Baltimore_Oriole', 'Altamira_Oriole', 'Audubons_Oriole'], ['Canvasback', 'American_Black_Duck', 'Bufflehead'], ['American_Wigeon', 'American_Black_Duck', 'Canada_Goose'], ['Audubons_Oriole', 'American_Goldfinch', 'Canada_Warbler'], ['Blue_Jay', 'Black_throated_Blue_Warbler', 'Black_necked_Stilt'], ['Altamira_Oriole', 'Bullocks_Oriole', 'Baltimore_Oriole'], ['Black_Turnstone', 'Brant', 'Anhinga'], ['California_Quail', 'Acorn_Woodpecker', 'American_Three_toed_Woodpecker'], ['Blue_Grosbeak', 'Bohemian_Waxwing', 'Cattle_Egret'], ['Black_backed_Woodpecker', 'Brown_Creeper', 'Black_Vulture'], ['Cassins_Kingbird', 'California_Towhee', 'Canyon_Towhee'], ['Altamira_Oriol

In [ ]:
import numpy as np

predicted_labels = []
true_labels = []

for images, labels in validation_dataset:
    preds = model.predict(images)
    pred_label_indices = np.argmax(preds, axis=1)
    
    true_label_indices = np.argmax(labels.numpy(), axis=1)
    
    predicted_labels.extend(pred_label_indices)
    true_labels.extend(true_label_indices)


predicted_label_names = [class_names[idx] for idx in predicted_labels]
true_label_names = [class_names[idx] for idx in true_labels]

In [13]:
count_matches = sum(1 for true, top3 in zip(true_label_names, t3labels) if true in top3)

print(f"Nombre de correspondances où le vrai label est dans le top 3: {count_matches}")

Nombre de correspondances où le vrai label est dans le top 3: 996


In [ ]:
import pandas as pd

results_df = pd.DataFrame({
    'True Label': true_label_names,
    'Predicted Label_1': t3labels
})

print(results_df)

In [16]:
f"The accuracy is: {model.evaluate(validation_dataset)[1]}"

27/27 [==============================] - 2s 56ms/step - loss: 3.4742 - accuracy: 0.3038


'The accuracy is: 0.3037974536418915'